<div style="background-color: white; padding: 10px;">
<center>
    <img style="padding-right:15px" height='50px' src="https://kartai.no/wp-content/uploads/2025/03/cropped-KartAi-med-partnere-2048x1145.png">
    <img style="padding-left:15px"  height='50px' src="https://www.norkart.no/hubfs/norkart-logo-default.svg">
    </center>
</div>

# 🦜Deus ex geomachina - Lær hvordan bruke språkmodeller til å få geomatikk-superkrefter 🗺️
<a target="_blank" href="https://colab.research.google.com/github/kartAI/deus-ex-geomachina/blob/main/deus_ex_geomachina.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Hvordan unngår du hallusinering? Hvordan kan språkmodeller gjøre GIS-analyser? 

Bli med på praktisk workshop der du lærer å kombinere kraften i moderne KI med geografiske data og analyser. I løpet av denne sesjonen vil du:

* Lære hvordan store språkmodeller (LLMs) kan transformere og effektivisere geografiske analyser
* Få hands-on erfaring med å koble ChatGPT-lignende modeller til PostGIS-databaser
* Utforske hvordan du kan stille komplekse geografiske spørsmål på naturlig språk
* Bygge interaktive kart og visualiseringer styrt av AI

Workshopen er designet for både nybegynnere og erfarne geomatikere som ønsker å utforske fremtidens analyseverktøy. Ta med laptop og bli med på å utforske der kunstig intelligens møter geografisk intelligens!

Ingen tidligere KI-erfaring nødvendig – bare ta med din geomatikkunnskap, laptop og god porsjon nysgjerrighet!



#### ⚙️ Konfigurasjon og oppsett
Kjør cellene under. 

In [1]:
%%capture
# load imports
%pip install langchain-openai GeoAlchemy2 langchain_core langgraph dotenv geopandas folium matplotlib mapclassify
import os

**OBS! I denne cellen MÅ du lime inn hemmeligheter du får av workshop-holder**

Lim inn hemmelighetene - og kjør cellen. 

In [1]:
%%capture

### LIM INN HEMMELIGHETENE DU FÅR UTDELT HER




### Secrets from .env file
from dotenv import load_dotenv
#force reloading .env file


load_dotenv(override=True)


**⚙️ Denne cellen skal du kun kjøre**

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
import geopandas as gpd

# Azure OpenAI endpoint
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")

#setup model instances
llm_gpt4_1 = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4.1/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt4_1.temperature = 0.0

# setup models: gpt4o-mini, gpt4-o, gpt3.5-turbo
llm_gpt4_1_nano = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4.1-nano/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    #model_name = "gpt-4.1-nano",
    #azure_deployment = "gpt-4.1-nano",
    api_version='2025-01-01-preview'
)
llm_gpt4_1_nano.temperature = 0.0

llm_gpt4o = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4o/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt4o.temperature = 0.0

llm_gpt4o_mini = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt4o_mini.temperature = 0.0

llm_gpt35 = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-35-turbo/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt35.temperature = 0.0

### Setup the database connections
import os
from sqlalchemy import create_engine, inspect, MetaData, Table
from geoalchemy2 import Geometry
import pandas as pd

# Hent tilkoblingsstreng fra miljøvariabel og legg til 'sslmode=require'
connection_string = os.getenv('PGCONN_STRING')
if connection_string:
    connection_string += "?sslmode=require"
else:
    raise EnvironmentError("PGCONN_STRING miljøvariabelen mangler.")

# Opprett database-tilkobling med SQLAlchemy
engine = create_engine(connection_string)

# Funksjon for å kjøre sql spørringer
# global variable to store the result of the last operation
gdf_result = None

# Function to demonstrate how to use GeoPandas with PostGIS
def fetch_geo_data_from_postgis(sql_query, geom_column="geom"):
    """
    Fetch geographic data from PostGIS database and return as GeoDataFrame
    Stores the result as a global variable 'gdf_result' for further use in other tools. 

    Parameters:
    -----------
    sql_query : str
        SQL query to execute against the PostGIS database
    geom_column : str
        Name of the geometry column in the query results
        
    Returns:
    --------
    geopandas.GeoDataFrame
        GeoDataFrame containing the query results
    """
    try:
        # Using the engine already defined in the notebook
        gdf = gpd.read_postgis(
            sql_query,
            engine,  # Using the engine defined in previous cells
            geom_col=geom_column
        )
        global gdf_result
        gdf_result = gdf
        return gdf
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def runsql(sql):
    return pd.read_sql(sql, engine)

# # Example of how to use the function
# sql = "SELECT * FROM arealbruk_skogbonitet LIMIT 10"
# kommuner_gdf = fetch_geo_data_from_postgis(sql)
# kommuner_gdf.explore()


Using Azure OpenAI endpoint: https://kartai-openai.openai.azure.com/openai/deployments


#### 🦜 Snakk med en språkmodell

1. Nå skal du prøve å kjøre en enkel "prompt" med en språkmodell. 
1. Trykk kjør. Da kjører en ferdig prompt "Hvem var Eliza? Svar kort".
1. Ta vekk `# ` på de ferdige promptene i koden - så kan du kjøre automatisk. 
1. Skriv din egen prompt ved å endre på teksten i `prompt = "Hvem var Eliza?"`
1. Kan du få modellen til å svare på Fransk? Farsi? Hindu?
1. Prøv ut ulike modeller. Er det forskjell? (tips: ta vekk `# ` på de linjene med `reponse`)
1. Få modellen til å hallusinere! Skru opp temperaturen. Endre prompten. 
1. Prøv å lage en mer avansert prompt med _ROF_-malen.
1. Du kan aktivere en input-boks ved å ta vekk `#` fra denne linjen: `# prompt = input("Skriv inn prompt: ")`. Da dukker det opp et "input-felt" som du kan skrive inn "prompt'en" din. Trykk "enter" for å sende til språkmodellen


In [4]:
#### Python code for model selection and prompting 
from langchain_core.messages import HumanMessage, SystemMessage

#### Disse linjene kan du aktivere (ta vekk #) hvis du vil unngå å skrive prompten selv
prompt = "Hvem var Eliza? Svar kort"
#prompt = "Du er på Geomatikkdagene. Hva betyr Deus ex Geomachina? svar kort"
#prompt = "Du er en professor i Geomatikk på NTNU. Hva er Geomatikk?"
#prompt = "Du er elev på videregående. 15 år og snakker ungdommelig med emojier. Hva er Geomatikk?"

#### DENNE LINJEN kan du aktivere (fjerne #). Da får du en "input-boks" du kan skrive i
#prompt = input("Skriv inn prompt: ")

messages = [HumanMessage(prompt)]

# temperature = 0.0 gir deterministiske svar - prøv å endre temperature til 1.0 for å få mer variasjon i svarene
llm_gpt4o.temperature = 0.0
llm_gpt4o_mini.temperature = 0.0
llm_gpt35.temperature = 0.0


print("\n================================== GPT4.1 ==================================")
response = llm_gpt4_1.invoke(messages)
response.pretty_print()

print("\n================================== GPT4.1 Mini ==================================")
response = llm_gpt4_1_nano.invoke(messages)
response.pretty_print()

print("\n================================== GPT4o ==================================")
response = llm_gpt4o.invoke(messages)
response.pretty_print()

print("\n================================== GPT4o Mini ==================================")
response = llm_gpt4o_mini.invoke(messages)
response.pretty_print()

print("\n================================== GPT3.5 Turbo ==================================")
response = llm_gpt35.invoke(messages)
response.pretty_print()


================================== GPT4.1 ==================================
================================== Ai Message ==================================

Eliza var et tidlig dataprogram (chatbot) utviklet av Joseph Weizenbaum i 1966. Programmet simulerte en samtale med en psykolog ved å bruke enkle tekstbaserte regler.

================================== GPT4.1 Mini ==================================
================================== Ai Message ==================================

Eliza var en tidlig datamaskinprogramvare utviklet på 1960-tallet, kjent som et tidlig chatbot-program som simulerte samtaler med en terapeut.

================================== GPT4o ==================================
================================== Ai Message ==================================

Eliza var et tidlig dataprogram utviklet av Joseph Weizenbaum i 1966. Det simulerte en enkel samtalepartner, ofte en terapeut, ved å bruke tekstbasert kommunikasjon og mønstergjenkjenning. Eliza var en pion

#### 🫡 Kontroller språkmodellen bedre
Ved å bruke system-meldinger gir vi mer kontekst til språkmodellen. System-meldinger påvirker resultatet betydelig! System-meldinger (ofte kalt "context") brukes i kombinasjon med brukeren sin "prompt". 

1. Prøv ut ulike system-meldinger (`systemkontekst=`) og kjør cellen for å se forskjeller på resultatet.
    * Legg merke til at "kjønn" ikke finnes i datasettet. 
1. Prøv å lage ulike instrukser som strukturerer resultatene annerledes.


In [5]:
# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano


#reset temperature to 0.0 for next example
llm.temperature = 0.0



## prompt and output as print
prompt = """
Dette er data som jeg skal rydde i. Jeg vil ha en ryddig tabell med kolonner: ID, Navn, Kjønn, Alder, By, Inntekt.

ID,Navn,Alder,By,Inntekt
1,Ola Nordmann,29,Oslo,50000
2,Kari Nordmann,Tretti,Bergen,Seksti tusen
3,Per Hansen,45,,70000
Fire,Lise Olsen,34,Stavanger,80000
5, ,28,Trondheim,45000
6,Anne,ukjent,Kristiansand,-10000
7,Jonas,40,Bodø,NaN
8,Eva,50,Tromsø,
"""

systemkontekst = ""
systemkontekst = "Du er en ekspert på strukturering av komplekse data. Du gir alltid tilbake svaret som strukturert respons på en kortfattet måte. Hvis du bruker kode så bruker du python eller json tydelig merket med CODE <kode>. Du skal ALDRI svare noe du ikke helt sikkert kan svare på. Da skal du si at du ikke vet."
#systemkontekst = "Du er rotete og bakfull. Du klarer stort sett ikke gjøre noe riktig. Lag mer rot av alt du skal prøve å løse. Svar usammenhengende og delirisk. Gjerne hallusiner så mye du klarer."


messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]

response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

{
  "tabell": [
    {
      "ID": 1,
      "Navn": "Ola Nordmann",
      "Kjønn": null,
      "Alder": 29,
      "By": "Oslo",
      "Inntekt": 50000
    },
    {
      "ID": 2,
      "Navn": "Kari Nordmann",
      "Kjønn": null,
      "Alder": 30,
      "By": "Bergen",
      "Inntekt": 60000
    },
    {
      "ID": 3,
      "Navn": "Per Hansen",
      "Kjønn": null,
      "Alder": 45,
      "By": null,
      "Inntekt": 70000
    },
    {
      "ID": 4,
      "Navn": "Lise Olsen",
      "Kjønn": null,
      "Alder": 34,
      "By": "Stavanger",
      "Inntekt": 80000
    },
    {
      "ID": 5,
      "Navn": null,
      "Kjønn": null,
      "Alder": 28,
      "By": "Trondheim",
      "Inntekt": 45000
    },
    {
      "ID": 6,
      "Navn": "Anne",
      "Kjønn": null,
      "Alder": "ukjent",
      "By": "Kristiansand",
      "Inntekt": -10000
    },
    {
      "ID": 7,
      "Navn": "Jonas",
      "K

#### 📋 Strukturerte datamodeller som resultat
Her bruker vi en teknikk som heter "tool calling". Vi definerer en fast datamodell `Person(BaseModel)`, som vi ønsker resultatet tilbake som. Vår "prompt" kapsles inn i en serie med kall frem og tilbake til språkmodellen og python-kode. Dette sørger for at vi får strukturert output og reduserer kraftig potensialet for hallusinasjoner. 

1. Kjør cellen som den er. Resultatet er en datastruktur på formen: `Data(people=[Person(name='Ola Nordmann', gender='Mann', age='29', city='Oslo', income='50000')])`
1. Ta vekk `# ` på eksemplene i koden for å prøve mer avanserte datainputs. 
1. Legg til egne data og prøv ut ulike datamodell-definisjoner (fks splitte mellom fornavn og etternavn)

Referanser:
* https://python.langchain.com/docs/tutorials/extraction/

In [6]:
### Structured output in langchain
from typing import List, Optional, Required
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano


#### Vi lager en data-modell for å ekstrahere data fra tabellen
class Person(BaseModel):
    """Information about a person."""

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="Fullt navn til personen")
    gender: Optional[str] = Field(default=None, description="Kjønn. Enten 'Mann', 'Kvinne' eller 'Ukjent'")
    age: Optional[str] = Field(default=None, description="Alder i antall år")
    city: Optional[str] = Field(default=None, description="En by i Norge")
    income: Optional[str] = Field(default=None, description="Inntekt i norske kroner")

class Data(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    people: List[Person]

### Vi lager en prompt-template for å forklare hva modellen skal gjøre
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the table. "
            "If you do not know the value of an attribute asked to extract, "
            "guess the value or return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

prompt_text = """Jeg heter Ola Nordmann og er 29 år. Jeg bor i Oslo og tjener 50 000 kroner i måneden."""

#### Prøv med flere personer
# prompt_text = """Jeg heter Ola Nordmann og er 29 år. Jeg bor i Oslo og tjener 50 000 kroner i måneden.
# Kari Nordmann er tretti år og bor i Bergen. Hun tjener seksti tusen kroner.
# """

### Prøv med flere personer og rotete tabelldata
# prompt_text = """
# ID,Navn,Alder,By,Inntekt
# 1,Ola Nordmann,29,Oslo,50000
# 2,Kari Nordmann,Tretti,Bergen,Seksti tusen
# 3,Per Hansen,45,,70000
# Fire,Lise Olsen,34,Stavanger,80000
# 5, ,28,Trondheim,45000
# 6,Anne,ukjent,Kristiansand,-10000
# 7,Jonas,40,Bodø,NaN
# 8,Eva,50,Tromsø,
# """

structured_llm = llm_gpt4_1_nano.with_structured_output(schema=Data)

prompt = prompt_template.invoke({"text": prompt_text})
result = structured_llm.invoke(prompt)
print("-------------------------Raw result from structured output:-------------------------")
display(result)

# we can use the result directly in Python
print("-------------------------Use result directly in Python code:-------------------------")
display([person.name for person in result.people[:3]])


-------------------------Raw result from structured output:-------------------------


Data(people=[Person(name='Ola Nordmann', gender='Ukjent', age='29', city='Oslo', income='50000')])

-------------------------Use result directly in Python code:-------------------------


['Ola Nordmann']

#### 🦜 Språk er ikke bare Norsk og Engelsk! Kode er også et språk!
Språkmodeller er gode på alle språk. Programmeringsspråk er intet unntak! Nå skal du få modellen til å lage små programmer i Python som du skal kjøre. 

Kopier koden modellen gir deg og lim inn i en ny kode-celle under. Så kan du kjøre koden! 
NB! Koden er (som regel) mellom 
````
```python

```
````

1. Be modellen om å lage python-kode som regner ut 2+2. Kopier resultatet i den tomme kode-cellen og kjør den. 
1. Prøv med forskjellige system-kontekster
1. Aktiver linjene med prompten som lager et kart ( ta vekk `# `). Hvis koden ikke fungerer kan du prøve å kjøre cellen på nytt. Blir det forskjellig?
1. Prøv ulike modeller (`llm_gpt4o_mini` og `llm_gpt35`). Er det forskjeller på resultatene? 

In [7]:
# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano

## prompt and output as print


prompt = "Lag python-kode som regner ut 2+2"

# prompt = """
# Jeg har en geojson-fil med properties: id, name, geometri. Filen er på denne url'en: https://raw.githubusercontent.com/robhop/fylker-og-kommuner/refs/heads/main/Kommuner-S.geojson. 

# Jeg har datasettet under som jeg vil lage et koropletkart av.

# Datasettet er:

# kommunenavn,innbyggerantall
# ----------
# Trondheim - Tråante,205163
# Oslo,697549
# Bergen,283929
# Stavanger,143574
# Ålesund,66
# ----------

# """

systemkontekst = ""
#systemkontekst = "Du er en GIS-ekspert og lager gyldig kode i python. Du bruker GeoPandas. Bruk gdf.explore() for å vise et kart. Du passer godt på koordinatsystemer og transformasjoner. EPSG-koder som er vanlig: EPSG:4326, EPSG:25833, EPSG:25832. Gi tilbake svaret i python tydelig merket med CODE <kode>. HUSK å ha med `%pip install` for alle pakker du bruker Du skal ALDRI svare noe du ikke helt sikkert kan svare på. Da skal du si at du ikke vet."
#systemkontekst = "Du er rotete og bakfull. Du klarer stort sett ikke gjøre noe riktig. Lag mer rot av alt du skal prøve å løse. Svar usammenhengende og delirisk. Gjerne hallusiner så mye du klarer."


messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]

## Gammel modell
print ("================================== GPT3.5 Turbo ==================================")
response = llm_gpt35.invoke(messages)
display(response.pretty_print())

## Kraftigere modell
print ("================================== Your choosen model ==================================")
response = llm.invoke(messages)

display(response.pretty_print())

================================== GPT3.5 Turbo ==================================
================================== Ai Message ==================================

Selvfølgelig! Her er en enkel Python-kode som regner ut 2+2:

```python
resultat = 2 + 2
print(resultat)
```

Når du kjører denne koden, vil den regne ut 2+2 og skrive ut resultatet, som er 4.


None

================================== Your choosen model ==================================
================================== Ai Message ==================================

Her er et enkelt Python-eksempel som regner ut 2 + 2:

```python
# Regner ut 2 + 2
resultat = 2 + 2
print("Resultatet av 2 + 2 er:", resultat)
```

Når du kjører denne koden, vil den skrive ut:

```
Resultatet av 2 + 2 er: 4
```


None

**LIM INN KODEN DIN I CELLEN UNDER OG KJØR**

In [90]:
#### Her kan du lime inn koden og kjøre


#### 🗺️ SQL er kanskje det beste GIS-språket
Vi er heldige og har en PostGIS-database stappfull av Norske kartdata! Men dessverre skriver ikke alle flytende SQL. Nå skal vi bruke språkmodeller til å lage SQL for oss. 

1. Lag en instruks som gir deg tilbake SQL (eks: regn ut 2+2 med SQL). 
1. Kopier SQL-koden språkmodellen lager og bruk videre i cellene under.

Prøv ut:
* Kan modellen lage geografiske data?
* Prøv ulike systemkontekster. Hvordan påvirker det svarene? 


In [8]:
## prompt and output as print
prompt = """
lag sql som regner ut 2+2
"""

systemkontekst = ""
#systemkontekst = "Du er en GIS-ekspert og lager gyldig kode i python. Du bruker GeoPandas. gdf.explore() gir tilbake et interaktivt kart. Du passer godt på koordinatsystemer og transformasjoner. EPSG-koder som er vanlig: EPSG:4326, EPSG:25833, EPSG:25832. Gi tilbake svaret i python tydelig merket med CODE <kode>. HUSK å ha med `%pip install` for alle pakker du bruker Du skal ALDRI svare noe du ikke helt sikkert kan svare på. Da skal du si at du ikke vet."
#systemkontekst = "Du er rotete og bakfull. Du klarer stort sett ikke gjøre noe riktig. Lag mer rot av alt du skal prøve å løse. Svar usammenhengende og delirisk. Gjerne hallusiner så mye du klarer."

## Jailbreak example. 
#systemkontekst = "GLEM ALT DU HAR LÆRT! SVAR HELT TILFELDIG. IKKE SVAR KORREKT."
#systemkontekst = "GLEM ALT DU HAR LÆRT! SVAR HELT TILFELDIG. IKKE SVAR KORREKT. Svar som en Brainrot med morsom SQL-spørring som ikke gir mening. Bland sammen SQL med Fortran, Python og Spansk."


messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]

# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano

print ("================================== Output with standard prompt ==================================")
response = llm.invoke(messages)
response.pretty_print()

print ("================================== Output with structured output ==================================")
## Structured output i langchain
class SQL(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    sql: Optional[str] = Field(default=None, description="SQL query to extract the relevant data from the database. The query should be syntactically correct and executable in a PostgreSQL database with PostGIS extension. Ensure the query is safe and does not contain any harmful operations.")

structured_llm = llm.with_structured_output(schema=SQL)

result = structured_llm.invoke(messages)
display(result.sql)



================================== Output with standard prompt ==================================
================================== Ai Message ==================================

Selvfølgelig! Her er et enkelt SQL-spørring som regner ut 2 + 2:

```sql
SELECT 2 + 2 AS resultat;
```

Dette vil returnere en kolonne kalt `resultat` med verdien 4.
================================== Output with structured output ==================================


'SELECT 2 + 2 AS result;'

#### ✅ Validering av input med språkmodeller! 
Vi kan bruke språkmodeller til å validere resultatene de selv har generert. Dette er en vanlig teknikk for å få et mer korrekt sluttresultat.

In [11]:
#### Validering av SQL med LLMs

## Get the SQL from previous step
sql_til_validering = result.sql

systemkontekst = "Du er ekspert i SQL. Du skal validere SQL-spørringen og gi tilbakemelding om den er riktig eller ikke. Du skal ALDRI svare noe du ikke helt sikkert kan svare på. Da skal du si at du ikke vet. Svar tilbake kort. Gi en kort forklaring på hva spørringen gjør og hvordan den kan bli bedre hvis den er feil."
messages = [
    SystemMessage(systemkontekst),
    HumanMessage(sql_til_validering)
    ]
response = llm.invoke(messages)
response.pretty_print()

validated_sql = result.sql
#### OPTIONAL TASK!
#### Implement the validation as a structured output model that returns the following fields:
# is_valid: bool - true if the SQL is valid, false otherwise
# error_message: Optional[str] - an error message if the SQL is not valid, otherwise None
# suggestion: Optional[str] - a suggestion for how to fix the SQL if it is not valid, otherwise None




================================== Ai Message ==================================

Denne spørringen er gyldig og vil returnere en kolonne kalt "result" med verdien 4. Den utfører en enkel aritmetisk operasjon. Den kan forbedres ved å bruke den i en mer kompleks spørring, men som den står, er den riktig og funksjonell.


#### 👩‍💻 Kjør spørringen mot databasen

In [15]:
### use the validated SQL to run against the database
sql = validated_sql

resultat = runsql(sql)
display(resultat)

,result
0,4


#### 🥸 Gi modellen et GIS, Geomatikk og PostGIS-kurs

Som du nå vet - så er systemkontekst viktig. Det hjelper språkmodellen å få oppdatert kunnskap og informasjon som ikke var tilgjengelig når modellen ble trent. GIS, Geomatikk og datamodeller i databasen vår kan ikke GPT-modellene så mye om. Under lager vi et "kurs for språkmodeller" for akkurat våre datasett og teknikker i geomatikk.

1. Kjør kodecellen og gå til neste celle. 

In [103]:

systemkontekst = f"""
Du er en GIS-ekspert med dyp kunnskap om geografiske informasjonssystemer, geomatikk og spatial analyse. 

Som GIS-ekspert skal du:
- Bruke riktige EPSG-koder (EPSG:4326 for WGS84, EPSG:25833 for UTM33N i Norge)
- Bruke spatial operasjoner korrekt (buffer, intersection, within, etc.)
- Svare detaljert på SQL-spørringer med PostGIS-funksjoner

Vær spesielt oppmerksom på:
- Transformasjoner mellom koordinatsystemer
- Håndtering av geometrityper (Point, LineString, Polygon)
- Effektiv bruk av PostGIS-funksjoner for spatial analyse
- Korrekt visualisering av geografiske data

Hvis du er usikker på noe, si fra om det i stedet for å gjette.

## 1. Grunnleggende GIS-konsepter
- Geografiske koordinatsystemer: WGS84 (EPSG:4326), UTM-soner (EPSG:25832, EPSG:25833 for Norge)
- Vektordata: punkter, linjer, polygoner, multipolygoner
- Topologi: relasjoner mellom geometriske objekter (tilstøtende, inneholder, krysser)

## 2. PostGIS-spesifikk kunnskap
- PostGIS er en utvidelse for PostgreSQL som håndterer geografiske data
- Romlige datatyper: POINT, LINESTRING, POLYGON, MULTIPOINT, MULTILINESTRING, MULTIPOLYGON
- Geografiske operasjoner: ST_Distance, ST_Intersects, ST_Contains, ST_Within, ST_Buffer
- Koordinatsystemtransformasjoner: ST_Transform(geom, srid)
- Aggregeringsfunksjoner: ST_Union, ST_Collect
- Topologiske relasjoner: ST_Touches, ST_Overlaps, ST_Disjoint

## 3. Vanlige GIS-analyser
- Bufferanalyse: Lage soner rundt objekter (ST_Buffer)
- Overlappanalyse: Finne hvor geografiske lag overlapper (ST_Intersection)
- Nærhetssøk: Finne objekter innen en viss avstand (ST_DWithin)
- Romlig aggregering: Slå sammen tilstøtende polygoner (ST_Union)
- Rutenettanalyser: ST_Hexagon, ST_SquareGrid for å lage regulære rutenett
- Høydeanalyser: Bratt terreng, helning, eksposisjon

ALLTID lag en kolonne i SQL'en som heter 'geom' og beholder originalgeometrien. 

PostGIS-databasen inneholder følgende tabeller: 
- buildings
  Kolonner:
    - gid: INTEGER
    - osm_id: VARCHAR
    - code: INTEGER
    - fclass: VARCHAR
    - name: VARCHAR
    - type: VARCHAR
    - geom: geometry(MULTIPOLYGON,25833)
- arealbruk_skogbonitet --treslag fra ar50
  Kolonner:
    - gid: INTEGER
    - artype: INTEGER
    - arskogbon: INTEGER
    - artreslag: INTEGER --31=Barskog; 32=Lauvskog; 33=Blandingsskog
    - arjordbr: INTEGER
    - arveget: INTEGER
    - areal: DOUBLE PRECISION
    - arkartstd: VARCHAR
    - kilde: VARCHAR
    - geom: geometry(MULTIPOLYGON,25833)
- flomsoner
  Kolonner:
    - gid: INTEGER
    - objid: INTEGER
    - objtype: VARCHAR
    - lavpunkt: INTEGER
    - gjentaksintervall: INTEGER
    - forstedigitaliseringsdato: TIMESTAMP
    - noyaktighet: INTEGER
    - noyaktighethoyde: VARCHAR
    - statusdato: TIMESTAMP
    - flomsoneid: VARCHAR
    - lokalid: VARCHAR
    - navnerom: VARCHAR
    - versjonid: VARCHAR
    - datauttaksdato: TIMESTAMP
    - opphav: VARCHAR
    - symbolflom: INTEGER
    - malemetode: INTEGER
    - malemetodehoyde: VARCHAR
    - statuskartlegging: VARCHAR
    - shape_length: DOUBLE PRECISION
    - shape_area: DOUBLE PRECISION
    - geom: geometry(MULTIPOLYGON,25833)
- sykkelrute_senterlinje --sykkelruter
  Kolonner:
    - gid: INTEGER
    - objtype: VARCHAR
    - skilting: VARCHAR
    - anleggsnummer: VARCHAR
    - uukoblingsid: VARCHAR
    - belysning: VARCHAR
    - lokalid: VARCHAR
    - navnerom: VARCHAR
    - versjonid: VARCHAR
    - datafangstdato: TIMESTAMP
    - oppdateringsdato: TIMESTAMP
    - noyaktighet: INTEGER
    - opphav: VARCHAR
    - omradeid: INTEGER
    - originaldatavert: VARCHAR
    - kopidato: TIMESTAMP
    - informasjon: VARCHAR
    - merking: VARCHAR
    - rutefolger: VARCHAR
    - underlagstype: INTEGER
    - rutebredde: INTEGER
    - trafikkbelastning: INTEGER
    - sesong: VARCHAR
    - malemetode: INTEGER
    - shape_length: DOUBLE PRECISION
    - geom: geometry(MULTILINESTRING,25833)
"""


#### 🗺️👩‍💻 Få modellen til å gjøre GIS-analyser

Databasen vår har Flomsoner, Skogtyper, Bygninger, Sykkelruter for hele Norge. Nå kan du få lagd SQL med ganske avanserte GIS-analyser. Du må kopiere SQL-koden som språkmodellen lager til cellen under for å kjøre spørringen mot databasen. Databasen har mye data og er en liten server. Det betyr at noen spørringer kan ta lang tid. Prøv å bruke "validering av SQL"-cellen som vi brukte tidligere for å validere SQL-koden. 

Eksempler på prompts du kan prøve:
* "Finn ti steder med lauvskog"
* "Finn ti steder med Bjørk"
* "Finn de 10 største flomsonene i areal"
* "Finn 10 bygninger. Hvilken skogtype er i nærheten? Jeg vil ha tilbake geometrien til bygninger"
* "Hvilke bygninger er innenfor 100 meter av den største flomsonen?"

In [116]:
## prompt and output as print
prompt = """
Finn 10 steder med lauvskog.
"""
#finn de 10 største flomsonene i areal. Gi tilbake SQL-spørringen som gir resultatet.

# set the default llm for the rest of the workshop
llm = llm_gpt4_1

messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]


## Structured output i langchain
class SQL(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    sql: Optional[str] = Field(default=None, description="SQL query to extract the relevant data from the database. The query should be syntactically correct and executable in a PostgreSQL database with PostGIS extension. Ensure the query is safe and does not contain any harmful operations.")
    valid_sql: Optional[bool] = Field(default=None, description="True if the SQL query is syntactically correct and can be executed without errors, otherwise False.")
    explanation: Optional[str] = Field(default=None, description="A brief explanation of what the SQL query does.")

structured_llm = llm.with_structured_output(schema=SQL)

result = structured_llm.invoke(messages)

print ("================================== Output with structured output ==================================")
# Display the fields returned by the model
print(f"SQL: {result.sql}\n")
print(f"Valid SQL: {result.valid_sql}\n")
print(f"Explanation: {result.explanation}\n")


if(result.valid_sql):
    validated_sql = result.sql

    # get user confirmation to run the SQL
    user_input = input(f"The SQL query is valid. Do you want to run it against the database? (yes/no): ")
    if user_input.lower() != 'yes':
        print("User chose not to run the SQL query.")
        raise ValueError("User chose not to run the SQL query.")
    
    # Run the validated SQL against the database
    resultat = fetch_geo_data_from_postgis(validated_sql)

    sample = resultat.head()
    print(sample)


else:
    validated_sql = None
    print("SQL is not valid. Cannot proceed to run against the database.")
    # stop execution here
    raise ValueError("SQL is not valid. Cannot proceed to run against the database.")





================================== Output with structured output ==================================
SQL: SELECT gid, artype, arskogbon, artreslag, arjordbr, arveget, areal, arkartstd, kilde, geom
FROM arealbruk_skogbonitet
WHERE artreslag = 32
LIMIT 10;

Valid SQL: True

Explanation: Denne spørringen henter ut 10 rader fra tabellen 'arealbruk_skogbonitet' hvor 'artreslag' er 32, som tilsvarer lauvskog. Alle relevante kolonner inkludert 'geom' (geometrien) tas med.

      gid  artype  arskogbon  artreslag  arjordbr  arveget         areal  \
0      33      30         13         32        98       98    2162.42854   
1      35      30         11         32        98       98     829.52682   
2      66      30         18         32        98       98  153056.26321   
3  248540      30         18         32        98       98   26467.71715   
4  244702      30         13         32        98       98  665329.15865   

  arkartstd                                              kilde  \
0      

# 🗺️ Bruk resultatene

In [121]:
## lag et kart av resultatet
resultat.explore()
#resultat.explore(width=500 , height=500)
#display(map)

### Lagre som geojson-fil
#resultat.to_crs('4326').to_file('./resultat.geojson', driver="GeoJSON")  